In [1]:
from functions import *


parameters = {
        'sample_rate': 1e6,  # Hz
        'num_samples': 1000,  # number of samples per call to rx()
        'center_freq': 2.35e9,  # Hz 2.4e9
        'bandwidth': 10,  # Hz
        'transmitter_attenuation': 0,  # dB
        'receiver_gain': 0,  # dB

        'size': 1.5,
        'num_iter': 10,
        'device_indices': [x+1 for x in range(20)],

        'time': time.strftime('%d%b%H%M', time.localtime())
    }


In [2]:
import plotly.graph_objects as go

def test_data_collection(tx):
    parameters['receiver_gain'] = 0
    signal = generate_signal()

    devices = init_devices(parameters)

    dataset = []

    for i in range(65):
        devices[tx].transmit(signal)
        receive_concurrently(devices=devices)
        data = []
        for j in range(20):
            data.append(devices[j].data)
            devices[j].sdr.rx_hardwaregain_chan0 = i
        dataset.append(data)

        devices[tx].stop_transmit()
        
    
    dataset = np.array(dataset)

    # np.save(os.path.join('result', f"{time.strftime('%d%b%H%M', time.localtime())}"), dataset)
    
    fig = go.Figure()

    for i in range(20):
        fig.add_trace(go.Scatter(
            y = np.abs(np.mean(dataset,axis=2)).transpose()[i],
            name = f"Pluto{i+1}"       # this sets its legend entry
        ))



    fig.update_layout(
        title=f"Pluto Saturation Test[Tx: Pluto{tx+1}]",
        xaxis_title="Receiver Gain(dB)",
        yaxis_title="Magnitude",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )

    fig.show()

    return dataset

gain_table = []
for tx in range(20):
    dataset = test_data_collection(tx)
    gain_list = []
    for rx in range(20):
        rx_data = (np.abs(np.mean(dataset,axis=2)).transpose()[rx])
        gain = np.where(rx_data == min(rx_data, key=lambda x:abs(x-1024)))[0].item()

        gain_list.append(gain)

    print(gain_list, end='\n')
    gain_table.append(gain_list)

print(gain_table)

[32, 32, 38, 34, 42, 37, 35, 34, 39, 39, 38, 35, 38, 36, 38, 39, 50, 47, 34, 24]


[31, 36, 37, 37, 43, 39, 37, 37, 36, 37, 34, 36, 34, 34, 37, 34, 41, 35, 27, 23]


[33, 39, 30, 38, 41, 37, 35, 36, 34, 42, 34, 34, 35, 36, 35, 36, 36, 38, 32, 31]


[37, 39, 42, 42, 38, 36, 56, 36, 34, 37, 35, 32, 35, 33, 33, 38, 36, 35, 32, 33]


[38, 47, 45, 45, 31, 30, 29, 35, 37, 44, 34, 33, 36, 32, 41, 37, 37, 39, 38, 36]


...

[37, 39, 37, 35, 35, 35, 34, 34, 36, 33, 38, 45, 44, 46, 37, 34, 34, 38, 46, 46]


[42, 38, 37, 41, 38, 34, 37, 33, 32, 33, 31, 37, 33, 46, 24, 39, 34, 35, 46, 57]


[40, 35, 38, 38, 38, 37, 34, 33, 33, 37, 33, 37, 33, 32, 35, 42, 37, 34, 39, 43]


[31, 31, 33, 33, 39, 33, 33, 33, 35, 35, 33, 35, 35, 44, 49, 55, 62, 47, 36, 47]


[27, 26, 32, 33, 36, 34, 33, 34, 34, 36, 33, 35, 44, 43, 39, 49, 57, 62, 51, 30]
[[32, 32, 38, 34, 42, 37, 35, 34, 39, 39, 38, 35, 38, 36, 38, 39, 50, 47, 34, 24], [31, 36, 37, 37, 43, 39, 37, 37, 36, 37, 34, 36, 34, 34, 37, 34, 41, 35, 27, 23], [33, 39, 30, 38, 41, 37, 35, 36, 34, 42, 34, 34, 35, 36, 35, 36, 36, 38, 32, 31], [37, 39, 42, 42, 38, 36, 56, 36, 34, 37, 35, 32, 35, 33, 33, 38, 36, 35, 32, 33], [38, 47, 45, 45, 31, 30, 29, 35, 37, 44, 34, 33, 36, 32, 41, 37, 37, 39, 38, 36], [36, 39, 44, 42, 27, 35, 29, 37, 34, 40, 35, 35, 34, 34, 34, 33, 35, 37, 33, 33], [33, 37, 35, 34, 23, 29, 35, 44, 38, 48, 40, 38, 39, 40, 39, 36, 35, 33, 33, 30], [33, 38, 34, 36, 36, 39, 44, 36, 34, 39, 37, 33, 36, 33, 37, 36, 35, 35, 32, 33], [35, 38, 32, 34, 38, 35, 38, 34, 59, 43, 47, 32, 33, 33, 38, 32, 34, 33, 35, 33], [41, 35, 40, 40, 44, 40, 51, 41, 38, 38, 36, 28, 32, 35, 41, 34, 32, 33, 32, 38], [33, 34, 36, 34, 35, 36, 47, 39, 39, 32, 51, 30, 34, 35, 42, 36, 35, 35, 34, 33], [40, 35, 34, 33,

In [3]:
gain_table = np.array(gain_table)

In [4]:
print(gain_table)

[[32 32 38 34 42 37 35 34 39 39 38 35 38 36 38 39 50 47 34 24]
 [31 36 37 37 43 39 37 37 36 37 34 36 34 34 37 34 41 35 27 23]
 [33 39 30 38 41 37 35 36 34 42 34 34 35 36 35 36 36 38 32 31]
 [37 39 42 42 38 36 56 36 34 37 35 32 35 33 33 38 36 35 32 33]
 [38 47 45 45 31 30 29 35 37 44 34 33 36 32 41 37 37 39 38 36]
 [36 39 44 42 27 35 29 37 34 40 35 35 34 34 34 33 35 37 33 33]
 [33 37 35 34 23 29 35 44 38 48 40 38 39 40 39 36 35 33 33 30]
 [33 38 34 36 36 39 44 36 34 39 37 33 36 33 37 36 35 35 32 33]
 [35 38 32 34 38 35 38 34 59 43 47 32 33 33 38 32 34 33 35 33]
 [41 35 40 40 44 40 51 41 38 38 36 28 32 35 41 34 32 33 32 38]
 [33 34 36 34 35 36 47 39 39 32 51 30 34 35 42 36 35 35 34 33]
 [40 35 34 33 35 34 45 31 31 26 33 34 40 40 43 40 37 36 37 40]
 [36 33 34 33 33 32 39 31 31 32 35 43 37 33 38 44 34 35 37 38]
 [35 33 34 32 34 33 36 32 33 35 37 50 34 34 37 35 34 33 46 48]
 [37 33 34 32 37 34 35 36 36 40 39 47 42 37 32 29 28 33 35 38]
 [37 39 37 35 35 35 34 34 36 33 38 45 44 46 37 34 34 38

In [7]:
np.save('result\\gain_table.npy', gain_table)